In [1]:
#import seaborn as sn
import wfdb
from wfdb import processing
import os
import glob
import pandas as pd
import numpy as np
#%matplotlib notebook
import matplotlib.pyplot as plt
from scipy import signal

In [19]:
### 128 ticks per second NOT 250
fs = 128
fs_target = 250


records_nsr = []
properties_nsr = []
annot_nsr = []
AnnSymb_nsr = []
AnnSamp_nsr = []
AnnRhythm_nsr = []
#Rpeak_Samp = []
#Rpeak_Symb = []
new_records_nsr = []
AnnSamp_250_nsr = []

for f in glob.glob('C:/Users/masud/Desktop/Github_Bachelor_Project/wfdb-python/normal-sinus-rhythm-data/*.dat'): ##### change the path (afib) to your own directory.
    sig, fields = wfdb.rdsamp(f[:-4], channels=[1])  #### In this function, you have the opportunity to pass "channels=[0]" or "channels=[1]" to select channel 1 or 2.
    
#    x, _ = processing.resample_sig(x=sig[:,0], fs=fields['fs'], fs_target=250)
#    new_sig = processing.resample_sig(x=sig, fs=128, fs_target=250)
#    new_sig = signal.resample(sig, 250)
    
    ann = wfdb.rdann(f[:-4], 'atr')
#    QRS = wfdb.rdann(f[:-4], 'qrs') ### xws extension instead
    Symb = pd.Series(ann.symbol)
    Samp = pd.Series(ann.sample)
#    x = processing.resample_sig(Samp, fs=fields['fs'], fs_target=250)
#    QRS_Symb = pd.Series(QRS.symbol)
#    QRS_Samp = pd.Series(QRS.sample)
    Rhythm = pd.Series(ann.aux_note)
    records_nsr.append(sig)
    properties_nsr.append(fields)
    annot_nsr.append(ann)
    AnnSymb_nsr.append(Symb)
    AnnSamp_nsr.append(Samp)
#    AnnSamp_250_nsr.append(x)
#    Rpeak_Symb.append(QRS_Symb)
#    Rpeak_Samp.append(QRS_Samp)
    AnnRhythm_nsr.append(Rhythm)
    
    

AnnSymb_nsr = pd.Series(AnnSymb_nsr).values
AnnSamp_nsr = pd.Series(AnnSamp_nsr).values

In [48]:
# print("Prints the last signal of the loop above")
# print(f"sig length:     {len(sig)}")
# print(f"new_sig length: {len(x)}")

In [35]:
# print(AnnRhythm_nsr[0])
# print("------------------------------------")
# print(AnnSamp_nsr[0])
# print("------------------------------------")
# print("All samples of R-peaks below")
# print(Rpeak_Samp[0])
# print("------------------------------------")
# print(Rpeak_Symb[0])
# print(records[0])
# print(type(records[0]))
# print(np.where(Rpeak_Samp[22] <= ))
# df.iloc[22] # acessing ROW 22
# print("------------------------------------")

In [47]:
#print(AnnSamp_nsr[0])
#print(AnnSamp_nsr[0]*2)

# Pre processing data...

In [36]:
labeled_Rpeaks_nsr = []
appended_data_nsr = []
for i in range(18): ## 18 records
    df = pd.DataFrame(AnnSamp_nsr[i]*2) ### Roughly resampling from 128 to 250 Hz
    appended_data_nsr.append(df)
        

labeled_Rpeaks_nsr = pd.concat(appended_data_nsr)
#labeled_Rpeaks_nsr

In [37]:
labeled_Rpeaks_nsr.rename(columns={0: 'Rpeaks'},
          inplace=True, errors='raise')
labeled_Rpeaks_nsr['Label'] = 0
labeled_Rpeaks_nsr

,Rpeaks,Label
0,2,0
1,104,0
2,258,0
3,414,0
4,568,0
...,...,...
111258,19446442,0
111259,19446610,0
111260,19446776,0
111261,19446938,0


In [38]:
Rpeaks_nsr = labeled_Rpeaks_nsr["Rpeaks"]
Label_nsr = labeled_Rpeaks_nsr["Label"]
Label_nsr = np.array(Label_nsr, dtype=np.float64)

#labeled_Rpeaks_N_AFIB[0:1000]
labeled_Rpeaks_nsr.isnull().values.any() ### Check for NaN values

False

### X

In [39]:
rri2 = np.diff(Rpeaks_nsr) ### Storing the intervals between rpeaks ### multiply by 2 here
print(rri2.dtype)
rri2 = np.array(rri2, dtype=np.float64) ### So no overflow issues happens when calc in for loop later
print(rri2.dtype)
print(len(rri2))
amount = -(len(rri2) % 20)
print(amount)
rec_amount = rri2[:amount]
print(f"Divisble by 20 --> {len(rec_amount)}")
segmented_rec = rec_amount.reshape(90339,20)
segmented_rec[0]

int32
float64
1806791
-11
Divisble by 20 --> 1806780


array([102., 154., 156., 154., 160., 156., 160., 152., 154., 158., 158.,
       166., 160., 158., 160., 166., 166., 164., 160., 162.])

In [40]:
features_nsr = []

    ### Calculating features for every single segmented "block" inside the segmented_record variable
for x in range(len(segmented_rec)):
    #MEAN
    ff1 = np.mean(segmented_rec[x])
    #STD
    ff2 = np.std(segmented_rec[x])
    #RMSSD
    sum_ = 0
    for y in range(len(segmented_rec[x]) - 1): ### loops 19 times
        sum_ += (segmented_rec[x][y] - segmented_rec[x][y+1])**2
    sum_multiplied = 1/(len(segmented_rec) - 1) * sum_
    ff3 = np.sqrt(sum_multiplied)
    #NORMALIZED RMSSD
    ff4 = (ff3 / ff1)

    my_features_nsr = pd.Series([np.around(ff1,3), np.around(ff2,3), np.around(ff3,3), np.around(ff4,3)], 
                            index=['Mean','STD','RMSSD','Normalized RMSSD'])
    features_nsr.append(my_features_nsr)

In [41]:
n = 0

print(f"Features for that specific segment:\n{features_nsr[n]}") 
print('\n'f"features_nsr variable contains {len(features_nsr)} segments")

Features for that specific segment:
Mean                156.300
STD                  13.104
RMSSD                 0.183
Normalized RMSSD      0.001
dtype: float64

features_nsr variable contains 90339 segments


### Y

In [42]:
y_amount = Label_nsr[:- 12] ### So it is divisible by 20
y_shape = y_amount.reshape(90339,20)
y_list = []

### Calc if every 20 segment block is Normal synus rythm or AFIB
for l in range(len(y_shape)):
    #y_segment = y_shape
    sum_segment = np.sum(y_shape[l])
    if sum_segment >= 10:
        sum_segment = 1
    else: sum_segment = 0
    y_list.append(sum_segment)

y_list = np.array(y_list, dtype=np.float64)
print(y_list.dtype)
print(f"Length of y_list: {len(y_list)}")

float64
Length of y_list: 90339


### X_train, X_test....

In [43]:
split_amount = int(90339 * 0.2)

In [44]:
y_test_nsr = np.asarray(y_list[:split_amount])
print(f"shape of y: {np.shape(y_test_nsr)}, type of y: {type(y_test_nsr)}")
X_test_nsr = np.asarray(features_nsr[:split_amount])
print(f"shape of X_test_nsr: {np.shape(X_test_nsr)}, type of X_test_nsr: {type(X_test_nsr)}")

shape of y: (18067,), type of y: <class 'numpy.ndarray'>
shape of X_test_nsr: (18067, 4), type of X_test_nsr: <class 'numpy.ndarray'>


In [3]:
# import os
# import subprocess #, shlex

# def cygwin(command):
#     """
#     Run a Bash command with Cygwin and return output.
#     """
#     # Find Cygwin binary directory
#     for cygwin_bin in [r'C:\cygwin\bin', r'C:\cygwin64\bin']:
#         if os.path.isdir(cygwin_bin):
#             break
#     else:
#         raise RuntimeError('Cygwin not found!')
#     # Make sure Cygwin binary directory in path
#     if cygwin_bin not in os.environ['PATH']:
#         os.environ['PATH'] += ';' + cygwin_bin
#     # Launch Bash
#     p = subprocess.Popen(
#         args=['bash', '-c', command],
#         stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#     p.wait()
#     # Raise exception if return code indicates error
#     if p.returncode != 0:
#         raise RuntimeError(p.stderr.read().rstrip())
#     # Remove trailing newline from output
#     return (p.stdout.read() + p.stderr.read()).rstrip()

In [43]:
# cygwin('pwd')

In [44]:
# cygwin('rdann -r 04936 -f 0 -a atr -v > data/04936_atr.txt')

# command_atr_new = "rdann -r 04936 -f 0 -a atr -v > /data/04936_atr.txt"
# command_atr = "rdann -r 04936 -f 0 -a atr -v > C:/Users/masud/Desktop/Github_Bachelor_Project/wfdb-python/data/04936_atr.txt"